In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) th
#at gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/options.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/financials.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/secondary_stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/trades.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/__init__.py
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_fin_spec.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/trades_spec.csv
/kaggle/input/jpx-tokyo-stock-

# Here starts my Baseline

Ideas:

1  - Feature engeneering- Basic and effective;

2 - Gradient Boosting Regressions;

3 - Neuronets Regressions; and

4 - Submission.



## 1 Feature Engeneering

In [2]:
def concat_df(df1, df2):
    df1 = pd.concat([df1, df2],
                    ignore_index=True, sort=False
                    ).drop_duplicates(["RowId"], keep="first")
    return df1

In [3]:
# For now... use pandas.... 
#If too heavy use Cudf

# For now... just fix the floats
dtypes = {
'SecuritiesCode' : np.int8,
'Open'           :     np.float16,
'High'           :     np.float16,
'Low'            :     np.float16,
'Close'          :     np.float16,
'Volume'         :       np.int8,
'AdjustmentFactor':    np.float16,
'ExpectedDividend':    np.float16,
'Target'        :      np.float16
}
path = "../input/jpx-tokyo-stock-exchange-prediction/"
df_prices = pd.read_csv(f"{path}train_files/stock_prices.csv",dtype = dtypes)
prices = pd.read_csv(f"{path}supplemental_files/stock_prices.csv",dtype = dtypes)
df_prices = concat_df(df_prices, prices)
df_prices = df_prices[df_prices.Date>"2021-11-01"]
df_prices.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273958 entries, 2288531 to 2562488
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   RowId             273958 non-null  object 
 1   Date              273958 non-null  object 
 2   SecuritiesCode    273958 non-null  int8   
 3   Open              273119 non-null  float16
 4   High              273119 non-null  float16
 5   Low               273119 non-null  float16
 6   Close             273119 non-null  float16
 7   Volume            273958 non-null  int8   
 8   AdjustmentFactor  273958 non-null  float16
 9   ExpectedDividend  2066 non-null    float16
 10  SupervisionFlag   273958 non-null  bool   
 11  Target            273952 non-null  float16
dtypes: bool(1), float16(7), int8(2), object(2)
memory usage: 10.7+ MB


In [4]:
# def prep_prices(price):
#     price.fillna(0,inplace=True)
#     return price

In [5]:
# df_prices = prep_prices(df_prices)
# pd.options.display.float_format = '{:,.6g}'.format
# df_prices.describe()

In [6]:
from datetime import datetime
import time

 # auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%Y-%m-%d").timetuple()))

# Data Features


In [7]:

# define function to compute log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)


# def fill_the_gaps(df):
#     new_df = pd.DataFrame(columns= df.columns)
#     for i in range(len(df['SecuritiesCode'].unique())):
#         new_df = new_df.append(df[df['SecuritiesCode'] == i].reindex(range(df[df['SecuritiesCode'] == i].index[0],df[df['SecuritiesCode'] == i].index[-1]+60,60),method='pad'))
#     new_df.fillna(method = 'pad', inplace = True) 
#     return new_df


def rsiFunc(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi



def get_features(df):

    df['log_return_5'] = log_return(df['Close'],periods=5)
    df['log_return'] = log_return(df['Close'],periods=1)

    upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
    lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

    df['upper_shadow'] = upper_shadow(df)
    df['lower_shadow'] = lower_shadow(df)
    
    df['EMA_21'] = df['Close'].ewm(span=21).mean()
    
    df['EMA_55'] = df['Close'].ewm(span=55).mean()
    
    df['EMA_315'] = df['Close'].ewm(span=315).mean()
    
    df['EMA_825'] = df['Close'].ewm(span=825).mean()
    
    window = 7
    
    no_of_std = 2
    
    df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
    df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
    df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
    df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
    window = 5
    
    df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
    df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
    df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
    df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
    df['MACD'] = df['EMA_7'] - df['EMA_5']
    
    
    df['rsi_5'] = rsiFunc(df['Close'].values, 5)
    
    df['rsi_7'] = rsiFunc(df['Close'].values, 7)
    
    df['rsi_21'] = rsiFunc(df['Close'].values, 21)

    
    
    #df = pd.concat([df, pd.get_dummies(df['Asset_ID'], prefix= 'Asset_')], axis=1)
    
    df[['log_return_5', 'log_return', 'upper_shadow', 'lower_shadow',
       'EMA_21', 'EMA_55', 'EMA_315', 'EMA_825', 'EMA_7', 'EMA_7_std',
       'EMA_7_BB_high', 'MA_7MA_BB_low', 'EMA_5', 'EMA_5_std', 'EMA_5_BB_high',
       'MA_5MA_BB_low', 'MACD', 'rsi_5', 'rsi_7', 'rsi_21']].astype('float16')
    
    return df



In [8]:
df_prices = get_features(df_prices)

df_prices = df_prices.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

df_prices = df_prices.fillna(0)

## Training a Simple Model

Xgboost

In [9]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
def model_training(X,y):
    # Model training
    
    model = xgb.XGBRegressor(n_estimators=5000,learning_rate=0.1 , tree_method = 'gpu_hist') 
    model.fit(X, y)
    
    return model


In [11]:
scaled_features = ['Open', 'High', 'Low', 'Close', 'Volume','log_return_5', 'log_return', 'upper_shadow', 'lower_shadow',
       'EMA_21', 'EMA_55', 'EMA_315', 'EMA_825', 'EMA_7', 'EMA_7_std',
       'EMA_7_BB_high', 'MA_7MA_BB_low', 'EMA_5', 'EMA_5_std', 'EMA_5_BB_high',
       'MA_5MA_BB_low', 'MACD', 'rsi_5', 'rsi_7', 'rsi_21']
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'log_return_5', 'log_return', 'upper_shadow', 'lower_shadow',
       'EMA_21', 'EMA_55', 'EMA_315', 'EMA_825', 'EMA_7', 'EMA_7_std',
       'EMA_7_BB_high', 'MA_7MA_BB_low', 'EMA_5', 'EMA_5_std', 'EMA_5_BB_high',
       'MA_5MA_BB_low', 'MACD', 'rsi_5', 'rsi_7', 'rsi_21']

In [12]:
# Creating endog and exog variables
y = df_prices[['Target']]
X = df_prices[features]

In [13]:
X[scaled_features] = scaler.fit_transform(X[scaled_features])


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [14]:
%%time

model_1 = model_training(X,y)

CPU times: user 24.6 s, sys: 667 ms, total: 25.3 s
Wall time: 26.9 s


In [15]:
model_1.score(X,y)

0.6166194256091879

In [16]:
import pickle

In [17]:
# Saving model

#Saving the sklearn model

filename = 'simple_xgb.sav'
pickle.dump(model_1, open(filename, 'wb'))

# save in text format
model_1.save_model("simple_xgb.txt")

# Submission


In [18]:
# def calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
#     """
#     Args:
#         df (pd.DataFrame): predicted results
#         portfolio_size (int): # of equities to buy/sell
#         toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
#     Returns:
#         (float): spread return
#     """
#     assert df['Rank'].min() == 0
#     assert df['Rank'].max() == len(df['Rank']) - 1
#     weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
#     purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
#     short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
#     return purchase - short

# def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
#     """
#     Args:
#         df (pd.DataFrame): predicted results
#         portfolio_size (int): # of equities to buy/sell
#         toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
#     Returns:
#         (float): sharpe ratio
#     """
#     buf = df.groupby('Date').apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
#     sharpe_ratio = buf.mean() / buf.std()
#     return sharpe_ratio, buf

def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

In [19]:
# # By Yuike - https://www.kaggle.com/code/ikeppyo/examples-of-higher-scores-than-perfect-predictions

# # This function adjusts the predictions so that the daily spread return approaches a certain value.
# def adjuster(df):
#     def calc_pred(df, x, y, z):
#         return df['Target'].where(df['Target'].abs() < x, df['Target'] * y + np.sign(df['Target']) * z)

#     def objective(trial, df):
#         x = trial.suggest_uniform('x', 0, 0.2)
#         y = trial.suggest_uniform('y', 0, 0.1)
#         z = trial.suggest_uniform('z', 0, 1e-3)
#         df["Rank"] = calc_pred(df, x, y, z).rank(ascending=False, method="first") - 1 
#         df["Rank"] = df["Rank"].astype("int")
#         return calc_spread_return_per_day(df, 200, 2)
    
#     def predictor_per_day(df):
#         study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=2022))
#         study.optimize(lambda trial: abs(objective(trial, df) - 10), 10)
#         return calc_pred(df, *study.best_params.values())

#     return df.groupby("Date").apply(predictor_per_day).reset_index(level=0, drop=True)

# def _predictor_base(feature_df):
#     return model.predict(feature_df[feats])

# def _predictor_with_adjuster(feature_df):
#     feature_df["Target"] = model.predict(feature_df[feats])
#     return adjuster(feature_df)

In [20]:
# predictor = _predictor_with_adjuster

In [21]:
import jpx_tokyo_market_prediction


env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    prices = get_features(prices)
    prices = prices.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
    prices = prices.fillna(0)
    prices[scaled_features] = scaler.fit_transform(prices[scaled_features])
    prices['pred'] = model_1.predict(prices[features])
    prices = add_rank(prices)
    feature_map = prices.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
